# 네이버 영화평 감성분석

In [ ]:
!pip install Konlpy #> dev/null # >dev/null : output 화면 미표기 맥북에서는 안되ㅏㄴ..? bㅠㅠ

In [11]:
import numpy as np
import pandas as pd 
import warnings 
warnings.filterwarnings('ignore')

In [32]:
# 네이버 영화 리뷰 데이터로 검색 
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt ', sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt ', sep='\t')

In [33]:
print(train_df.shape, test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리 


- train dataset

In [34]:
# Null 데이터 확인 
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [35]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [36]:
# 중복 여부 확인 
train_df.document.nunique()
# 데이터 수:149995, 고유값:146182 서로 달라 -> 중복 존재!

146182

In [37]:
# 중복 데이터 제거 
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [38]:
# 긍정(1)/부정(0) 값 분포 - df.label
train_df.label.value_counts()
# 대략적으로 점수-> 10-긍정, 9~7-중립, 0~6-부정
#-> 부정이 더 많아 

0    73342
1    72840
Name: label, dtype: int64

- Test dataset

In [39]:
# Null 데이터 확인 
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [40]:
# Null 데이터 제거
test_df.dropna(how='any', inplace=True)
test_df.shape

(49997, 3)

In [41]:
# 중복 여부 확인 
test_df.document.nunique() 
# 중복 존재!!

49157

In [42]:
# 중복 데이터 제거 
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [43]:
# 긍정(1)/부정(0) 값 분포 - df.label
test_df.label.value_counts()
#-> 긍정이 더 많아

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리

- Train dataset

In [45]:
# 한글 이외의 문자는 공배긍로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎ가-힣ㅏ-ㅣ]', ' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [46]:
# ''만 남은 데이터 --> np.nan으로 대체한 후 제거 
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [47]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- Test dataset

In [49]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎ가-힣ㅏ-ㅣ]', ' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape


(48852, 3)

- 전처리 끝난 데이터 저장 

In [50]:
train_df.to_csv('naver_movie_train_전처리완료.tsv', sep='\t', index=False)
test_df.to_csv('naver_movie_test_전처리완료.tsv', sep='\t', index=False)

### 3. 한글 처리

In [77]:
from konlpy.tag import Okt
okt = Okt()
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text) # 한국어 형태소 분리

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [78]:
okt.morphs(text, stem=True)
# string 형태소 분석 -> 리스트 형태
# Countervectorizer([text]) 안에 리스트 형식으로 들어가해

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [79]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
' '.join([word for word in okt.morphs(text, stem=True) if word not in stopwords])
# stem=True: 형태소 단위로 나눈 후 어간을 추출
# 한글 불용어 리스트 찾아보면 있지만 ,, 정확도는 ,,ㅎㅎ 

'교도소 이야기 구먼 솔직하다 재미 없다 평점 조정'

In [80]:
from tqdm.notebook import tqdm
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']

X_train = []
for sentence in tqdm(train_df.document):
  morphs = okt.morphs(sentence, stem=True)
  tmp_str = ' '.join([word for word in morphs if word not in stopwords])
  X_train.append(tmp_str)
#X_train

  0%|          | 0/145393 [00:00<?, ?it/s]

In [81]:
X_test= []
for sentence in tqdm(test_df.document):
  morphs = okt.morphs(sentence, stem=True)
  tmp_str = ' '.join([word for word in morphs if word not in stopwords])
  X_test.append(tmp_str)


  0%|          | 0/48852 [00:00<?, ?it/s]

In [61]:
# y_train / y_test 데이터 생성 
y_train = train_df.label.values # numpy array 구조
y_test = test_df.label.values

### 4. feature 변환 + 모델 학습/평가
- Pipeline 활용

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression         
from sklearn.pipeline import Pipeline

In [63]:
cvect = CountVectorizer()
lr = LogisticRegression(random_state=2022)
pipeline = Pipeline([('CVECT', cvect), ('LR', lr)])

In [64]:
%time pipeline.fit(X_train, y_train) # 학습 

CPU times: user 1.99 s, sys: 24.9 ms, total: 2.02 s
Wall time: 2.22 s


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [65]:
pipeline.score(X_test, y_test) # 평가
# 강사님 결과: 0.8273356259723246 -> 코드는 똑같은거 같은데..?

0.4980758208466388

### 5. 실제 데이터 테스트


In [66]:
review1 = '모든 국민이 봤으면 하는 영화입니다.'
review2 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.'

In [67]:
# 전처리 
import re
review1 = re.sub('[^가-힣]', ' ', review1)
review2 = re.sub('[^가-힣]', ' ', review2)

In [75]:
# review1에 대한 
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join([word for word in morphs if word not in stopwords])
# review2에 대한 
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join([word for word in morphs if word not in stopwords])

In [76]:
pipeline.predict([review1, review2])
# 강사님 결과값은 [1,0]인데..?

array([0, 0])

### 6. 최적의 파라미터 찾기 (교차검증)

In [72]:
from sklearn.model_selection import GridSearchCV
params={
    'CVECT__ngram_range':[(1,1), (1,2)],
    'CVECT__max_df': [0.95, 0.98],
    'LR__C': [1, 5],
}

In [73]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1)
%time grid_pipe.fit(X_train, y_train)

ValueError: ignored

In [ ]:
grid_pipe.best_params_

In [ ]:
grid_pipe.best_estimate_.score(X_test, y_test)
# ngram_range (1, 2)로 하여 성능 더 좋아졌다! 